 - Step 0: Label all client nodes as “not served”. Go to Step 1.
 - Step 1: If all client nodes are already labelled “served”, then STOP. Otherwise, start at one of the centres (or ‘depots’) with a new staff member, initialize the work duration of this staff member as zero, and go to Step 2.
 - Step 2: From the current node, pick the nearest neighbour client node j that is “not served”. If the current work duration and the walking distance to j add up to more than 7 hours, then this staff member has completed their shift, go to Step 1. Otherwise, go to Step 3.
 - Step 3: Add the walking distance to j as well as the task duration at j to the current work duration. Update the label of node j from “not served” to “served”. If the updated work duration is more than 7 hours, then this staff member has completed their shift, go to Step 1. Otherwise, go to Step 2.

Depots are 71, 142, 280, 3451, 6846, and 7649

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import csv
import itertools
import copy
import networkx as nx
import scipy as sci

In [2]:
CareDist_Matrix = np.loadtxt(open("CareDistances-FULL.csv", "rb"), dtype=int, delimiter=",", skiprows=1)
CareDist_Matrix

array([[   0,    4,   38, ..., 8360, 8507, 8545],
       [   4,    0, 1280, ...,  777, 1025, 1017],
       [  38, 1280,    0, ..., 1408, 1515, 1433],
       ...,
       [8360,  777, 1408, ...,    0, 1530, 1449],
       [8507, 1025, 1515, ..., 1530,    0,   82],
       [8545, 1017, 1433, ..., 1449,   82,    0]])

Replace all 0 distance values with a large number to prevent routing there due to the 0 distance

In [3]:
CareDist_Matrix[CareDist_Matrix == 0] = 10000000

In [4]:
CareDist_Matrix[0,0]=0

# Create the Data
The code below creates the data for the problem.

In [5]:
def create_data_model():
    """Stores the data for the problem."""
    data = {}
    data['distance_matrix'] = CareDist_Matrix # yapf: disable
    data['num_vehicles'] = 2
    data['depot'] = 71
    return data

The distance matrix is an array whose i, j entry is the distance from location i to location j in miles, where the array indices correspond to the locations

Note: The order of the locations in the distance matrix is arbitrary, and is unrelated to the order of locations in any solution to the TSP.

The data also includes:

 - The number of vehicles in the problem, which is 1 because this is a TSP. (For a vehicle routing problem (VRP), the number of vehicles can be greater than 1.)
 - The depot: the start and end location for the route. In this case, the depot is 71, which is one of 7 depotsbm.

# Create the routing model
The following code in the main section of the programs creates the index manager (manager) and the routing model (routing). 

The method manager.IndexToNode converts the solver's internal indices (which you can safely ignore) to the numbers for locations. 

Location numbers correspond to the indices for the distance matrix.

In [6]:
from ortools.constraint_solver import pywrapcp

data = create_data_model()
manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                       data['num_vehicles'], data['depot'])
routing = pywrapcp.RoutingModel(manager)

The inputs to RoutingIndexManager are:

 - The number of rows of the distance matrix, which is the number of locations (including the depot).
 - The number of vehicles in the problem.
 - The node corresponding to the depot.

# Create the distance callback
To use the routing solver, you need to create a distance (or transit) callback: a function that takes any pair of locations and returns the distance between them. The easiest way to do this is using the distance matrix.

The following function creates the callback and registers it with the solver as transit_callback_index.

In [7]:
def distance_callback(from_index, to_index):
    """Returns the distance between the two nodes."""
    # Convert from routing variable Index to distance matrix NodeIndex.
    from_node = manager.IndexToNode(from_index)
    to_node = manager.IndexToNode(to_index)
    return data['distance_matrix'][from_node][to_node]

transit_callback_index = routing.RegisterTransitCallback(distance_callback)
routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

The callback accepts two indices, from_index and to_index, and returns the corresponding entry of the distance matrix.

# Add a distance dimension
To solve this VRP, you need to create a distance dimension, which computes the cumulative distance traveled by each vehicle along its route. You can then set a cost proportional to the maximum of the total distances along each route. Routing programs use dimensions to keep track of quantities that accumulate over a vehicle's route. See Dimensions for more details.

The following code creates the distance dimension, using the solver's AddDimension method. The argument transit_callback_index is the index for the distance_callback.

In [8]:
# Max time is 7 hours which is 25200 seconds or 420 minutes

dimension_name = 'Distance'
routing.AddDimension(
    transit_callback_index,
    0,  # no slack
    25200,  # vehicle maximum travel distance
    True,  # start cumul to zero
    dimension_name)
distance_dimension = routing.GetDimensionOrDie(dimension_name)
distance_dimension.SetGlobalSpanCostCoefficient(100)

The method SetGlobalSpanCostCoefficient sets a large coefficient (100) for the global span of the routes, which in this example is the maximum of the distances of the routes. This makes the global span the predominant factor in the objective function, so the program minimizes the length of the longest route.

# Set search parameters
The following code sets the default search parameters and a heuristic method for finding the first solution:

In [9]:
from ortools.constraint_solver import routing_enums_pb2

search_parameters = pywrapcp.DefaultRoutingSearchParameters()
search_parameters.first_solution_strategy = (
    routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)

The code sets the first solution strategy to PATH_CHEAPEST_ARC, which creates an initial route for the solver by repeatedly adding edges with the least weight that don't lead to a previously visited node (other than the depot). For other options, see [First solution strategy.](https://developers.google.com/optimization/routing/routing_options#first_sol_options)

# Add the solution printer
The function that displays the solution returned by the solver is shown below. The function extracts the route from the solution and prints it to the console.

In [10]:
def print_solution(data, manager, routing, solution):
    """Prints solution on console."""
    max_route_distance = 0
    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        plan_output = 'Route for vehicle {}:\n'.format(vehicle_id)
        route_distance = 0
        while not routing.IsEnd(index):
            plan_output += ' {} -> '.format(manager.IndexToNode(index))
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(
                previous_index, index, vehicle_id)
        plan_output += '{}\n'.format(manager.IndexToNode(index))
        plan_output += 'Distance of the route: {}m\n'.format(route_distance)
        print(plan_output)
        max_route_distance = max(route_distance, max_route_distance)
    print('Maximum of the route distances: {}m'.format(max_route_distance))

The function displays the optimal route and its distance, which is given by ObjectiveValue().

# Solve and print the solution
Finally, you can call the solver and print the solution:

In [ ]:
solution = routing.SolveWithParameters(search_parameters)
if solution:
    print_solution(manager, routing, solution)

This returns the solution and displays the optimal route.

# Save routes to a list or array
As an alternative to printing the solution directly, you can save the route (or routes, for a VRP) to a list or array. This has the advantage of making the routes available in case you want to do something with them later. For example, you could run the program several times with different parameters and save the routes in the returned solutions to a file for comparison.

The following functions save the routes in the solution to any VRP (possibly with multiple vehicles) as a list (Python)

In [ ]:
def get_routes(solution, routing, manager):
  """Get vehicle routes from a solution and store them in an array."""
  # Get vehicle routes and store them in a two dimensional array whose
  # i,j entry is the jth location visited by vehicle i along its route.
  routes = []
  for route_nbr in range(routing.vehicles()):
    index = routing.Start(route_nbr)
    route = [manager.IndexToNode(index)]
    while not routing.IsEnd(index):
      index = solution.Value(routing.NextVar(index))
      route.append(manager.IndexToNode(index))
    routes.append(route)
  return routes

You can use these functions to get the routes in any of the VRP examples in the Routing section.

The following code displays the routes.

In [ ]:
routes = get_routes(solution, routing, manager)
# Display the routes.
for i, route in enumerate(routes):
  print('Route', i, route)